In [3]:
import torch
import random
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# read all words

words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
len(words)

32033

Vocab

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0

itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [7]:
block_size = 3

def build_dataset(words):

    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] #crop and append

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y


random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
X_train, Y_train = build_dataset(words[:n1])
X_dev, Y_dev = build_dataset(words[n1:n2])
X_test, Y_test = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


Utility func to compare grads

In [8]:
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')
    


In [16]:
n_embed = 10 #dim of the char embeddings
n_hidden = 64 #num of neurons in the hidden layer of MLP

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed), generator=g)

#Layer1
W1 = torch.randn((block_size*n_embed, n_hidden), generator=g) * (5/3)/((n_embed*block_size)**0.5) #kaining init
b1 = torch.randn(n_hidden, generator=g) * 0.1 #bias is optional here since we use batchnorm after this which cancels out bias term

#Layer2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

#BatchNorm Params
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))* 0.1


parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True


4137


In [10]:
batch_size = 32
n = batch_size

#construct a minibatch
ix = torch.randint(0, X_train.shape[0], (batch_size, ), generator=g)
Xb, Yb = X_train[ix], Y_train[ix]

In [17]:
#forward pass, split into small steps

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

#Linear Layer 1
h_pre_bn = embcat @ W1 + b1 #hidden layer preact

#Batchnorm layer
bnmeani = 1/n * h_pre_bn.sum(0, keepdim=True)
bndiff = h_pre_bn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1) * (bndiff2).sum(0, keepdim=True) #Bessel's correctn, div by n-1 not n
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
h_pre_act = bngain * bnraw + bnbias

#Non linearity
h = torch.tanh(h_pre_act) #hidden layer

#Linear Layer 2
logits = h @ W2 + b2 #output layer

#cross entropy loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes #subs max for stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum**-1 #if 1/counts_sum is used, backprop wont be exact
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

#Pytorch backward pass
for p in parameters:
    p.grad = None

for t in [emb, embcat, h_pre_bn, bnmeani, bndiff, bndiff2, bnvar, 
          bnvar_inv, bnraw, h_pre_act, h, logits, logit_maxes, norm_logits, counts, 
          counts_sum, counts_sum_inv, probs, logprobs, loss]:
    t.retain_grad()

loss.backward()
loss

tensor(3.3528, grad_fn=<NegBackward0>)

1. Backpropagate manually

- Cal grads of each of the intermediate variables

In [18]:
#dlogprobs = d(Loss)/dlogprobs

#loss = -(a + b + c)/3 ==> dloss/da = -1/3

#a, b, c in logprobs participate in cal loss for rest grad will be zero

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n # n is the batchsize here, from each batch one prob contributes to loss, as indexed by Yb

#check with pytorch
cmp('logprobs', dlogprobs, logprobs)



logprobs        | exact: True  | approximate: True  | maxdiff: 0.0


In [19]:
#dprobs = local der * dlogprobs -> 1/x * dlogprobs

dprobs = (1.0/probs) * dlogprobs
cmp('probs', dprobs, probs)

probs           | exact: True  | approximate: True  | maxdiff: 0.0


- dprobs will be boosted if probs is low and dlogprobs is non zero, which means cases where correct label has low prob

In [22]:
counts.shape, counts_sum_inv.shape

(torch.Size([32, 27]), torch.Size([32, 1]))

In [24]:
dprobs.shape

torch.Size([32, 27])

In [23]:
#count_sum_inv is broadcasted to mul with counts

In [38]:
#dcounts = count_sum_inv * dprobs 
#dcounts_sum_inv = sum(counts * dprobs, 1) : also since this is broadcasted to 27 cols, we sum the grad

dcounts1 = counts_sum_inv * dprobs
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)

cmp('counts', dcounts1, counts)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

counts          | exact: False | approximate: False | maxdiff: 0.003835804760456085
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0


- dcounts does not match here since it is being used twice, to cal count_sum also, so grad will flow in from there as well

In [37]:
#dcount_sum = -counts_sum**-2 * dcounts_sum_inv

dcounts_sum = -counts_sum**-2 * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0


In [39]:
#2nd part of dcounts = 1 * dcounts_sum, grad dlows equally to all elements of the sum

dcounts2 = 1 * dcounts_sum
dcounts = dcounts1 + dcounts2
cmp('counts', dcounts, counts)

counts          | exact: True  | approximate: True  | maxdiff: 0.0


In [41]:
dcounts1.shape, dcounts2.shape

(torch.Size([32, 27]), torch.Size([32, 1]))